In [66]:
# Dependencies
import pandas as pd

In [67]:
# Load & read in geocodes file
data_load = "../RawData/all-geocodes-v2019.csv"
raw_data = pd.read_csv(data_load, header=4,  encoding = 'unicode_escape')
raw_data.head()

,Summary Level,State Code (FIPS),County Code (FIPS),County Subdivision Code (FIPS),Place Code (FIPS),Consolidtated City Code (FIPS),Area Name (including legal/statistical area description)
0,10,0,'000,0,0,0,United States
1,40,1,'000,0,0,0,Alabama
2,50,1,'001,0,0,0,Autauga County
3,50,1,'003,0,0,0,Baldwin County
4,50,1,'005,0,0,0,Barbour County


In [68]:
# drop unwanted columns
raw_data = pd.DataFrame(raw_data, columns=['Summary Level', 'State Code (FIPS)', 'County Code (FIPS)', 'Area Name (including legal/statistical area description)'])

raw_data.head()

,Summary Level,State Code (FIPS),County Code (FIPS),Area Name (including legal/statistical area description)
0,10,0,'000,United States
1,40,1,'000,Alabama
2,50,1,'001,Autauga County
3,50,1,'003,Baldwin County
4,50,1,'005,Barbour County


In [69]:
# slicing off ' from county code
raw_data['County Code (FIPS)'] = raw_data['County Code (FIPS)'].str[1:]

raw_data.head()

,Summary Level,State Code (FIPS),County Code (FIPS),Area Name (including legal/statistical area description)
0,10,0,000,United States
1,40,1,000,Alabama
2,50,1,001,Autauga County
3,50,1,003,Baldwin County
4,50,1,005,Barbour County


In [70]:
# Concatinate state and county code
raw_data['combined fips'] = raw_data['State Code (FIPS)'].map(str) + raw_data['County Code (FIPS)']
raw_data.head()

,Summary Level,State Code (FIPS),County Code (FIPS),Area Name (including legal/statistical area description),combined fips
0,10,0,000,United States,0000
1,40,1,000,Alabama,1000
2,50,1,001,Autauga County,1001
3,50,1,003,Baldwin County,1003
4,50,1,005,Barbour County,1005


In [ ]:
# Create state table - filter
